<a href="https://colab.research.google.com/github/Insights-Labs-Consultant-Agency/yelp-google-maps-reviews-and-recommendations/blob/data-pipeline/notebooks/4.3-fp-etl-glue-job-yelp-checkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción, transformación y carga (ETL)

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos de Yelp y Google Maps utilizando la librería AWS Glue 3.0 que servirá como base para los scripts de los diferentes ETL Glue Jobs que se usaran en AWS Glue Workflow en el proceso de carga al DW. En esta etapa, se realizará un proceso de limpieza previa y posterior normalización para construir un DER.

## 0 Configuraciones Globales e Importaciones

En esta sección,instalamos e importamos todas las librerías y/o módulos necesarios para nuestro proceso ETL y establecemos configuraciones globales de ser requerido.

### Instalación de librerías y/o Dependencias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://aws-glue-etl-artifacts.s3.amazonaws.com/glue-common/apache-maven-3.6.0-bin.tar.gz
!tar xvf apache-maven-3.6.0-bin.tar.gz -C /bin/ > /dev/null
!wget -q https://aws-glue-etl-artifacts.s3.amazonaws.com/glue-3.0/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3.tgz
!tar xvf spark-3.1.1-amzn-0-bin-3.2.1-amzn-3.tgz -C /bin/ > /dev/null
!pip install -q findspark

### Exportación de Variables Entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] += ":/bin/apache-maven-3.6.0/bin"
os.environ["SPARK_HOME"] = "/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/"
os.environ["SPARK_CONF_DIR"] = "/bin/aws-glue-libs/conf"

### Instalación de AWS GLue 3.0 Libs

In [ ]:
!git clone -b glue-3.0 https://github.com/awslabs/aws-glue-libs.git /bin/aws-glue-libs
!chmod +x /bin/aws-glue-libs/bin/glue-setup.sh
!bash /bin/aws-glue-libs/bin/glue-setup.sh > /dev/null
!cp -r /bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/jars/netty-all-4.1.51.Final.jar /bin/aws-glue-libs/jarsv1/

Cloning into '/bin/aws-glue-libs'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 321 (delta 63), reused 62 (delta 45), pack-reused 222
Receiving objects: 100% (321/321), 163.93 KiB | 4.43 MiB/s, done.
Resolving deltas: 100% (203/203), done.
rm: cannot remove 'PyGlue.zip': No such file or directory
rm: cannot remove '/bin/aws-glue-libs/conf/spark-defaults.conf': No such file or directory


### Importación de Librerías y/o Módulos

In [ ]:
import sys
sys.path.extend(["/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/python","/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/python/lib/py4j-0.10.9-src.zip","/bin/aws-glue-libs/PyGlue.zip"])

import findspark
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import split, explode, monotonically_increasing_id

findspark.init()
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark

## 1 Extracción

En esta sección, extraemos los datasets de la fuente y los leemos como un DataFrame de PySpark.

In [ ]:
# Ruta al archivo JSON checkin
file_path = '/content/drive/MyDrive/data/raw/yelp/checkin.json'

In [ ]:
# Lee el archivo JSON localmente con PySpark
df = spark.read.json(file_path)

## 2 Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas  la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

In [ ]:
# Divide la columna 'date' en una lista de fechas
df = df.withColumn('date', split(df['date'], ', '))

In [ ]:
# Convierte la lista de fechas en filas separadas
df = df.withColumn('date', explode(df['date']))

In [ ]:
# Convierte la columna 'date' al formato datetime
df = df.withColumn('date', df['date'].cast('timestamp'))

In [ ]:
# Ordena el DataFrame por 'business_id' y 'date'
df = df.orderBy('date')

In [ ]:
# Agrega una columna 'checkin_id' con un ID único para cada registro
df = df.withColumn('checkin_id', monotonically_increasing_id())

In [ ]:
# Reordena las columnas
df = df.select('checkin_id', 'business_id', 'date')

In [ ]:
df.show()

+----------+--------------------+-------------------+
|checkin_id|         business_id|               date|
+----------+--------------------+-------------------+
|         0|jm_6bIhR_TuciFIk5...|2009-12-30 02:53:27|
|         1|yEKmlxvhA2EWrMLbw...|2009-12-31 00:18:24|
|         2|TRxR32_T_7Ly_RW4K...|2010-01-16 00:36:59|
|         3|OzoE9NH0xjhfsVMc7...|2010-01-16 01:15:27|
|         4|7lwe7n-Yc-V9E_HfL...|2010-01-16 02:12:05|
|         5|qDheewhaZNDWLfpsr...|2010-01-16 02:12:19|
|         6|QYni15AsaQ8F20jEk...|2010-01-16 02:25:21|
|         7|cdIZWzfLzU8AAnS0r...|2010-01-16 02:56:02|
|         8|frkdRWMpN70bO3jFW...|2010-01-16 03:45:04|
|         9|32exKYuOUcXARWrhv...|2010-01-16 03:52:21|
|        10|RHdIW62cidKmRIjxM...|2010-01-16 03:56:53|
|        11|pDyhy6j_QMhP69wLm...|2010-01-16 04:08:09|
|        12|s2JiYzE7i68cXIV6Y...|2010-01-16 04:14:00|
|        13|aM3gFY8Eq2KJuvNBl...|2010-01-16 04:21:18|
|        14|PY9GRfzr4nTZeINf3...|2010-01-16 05:02:18|
|        15|DQOLYljad7wQnMsS

## 3. Carga

Finalmente, en esta sección cargamos nuestros datos transformados en formato parquet a su destino correspondiente.

### Google Drive

In [ ]:
# Ruta al archivo Parquet local
file_path = '/content/drive/MyDrive/data/cleaned/yelp/checkin.parquet'

# Escribe el DataFrame a un archivo Parquet localmente
df.write.parquet(file_path)


### S3

In [ ]:
# Convierte el DataFrame de Spark a un DynamicFrame de Glue
dyf = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")

In [ ]:
# Escribe el DynamicFrame a S3 en formato Parquet
glueContext.write_dynamic_frame.from_options(
    frame = dyf,
    connection_type = "s3",
    connection_options = {"path": "s3://ruta/al/bucket"},
    format = "parquet"
)